<a href="https://colab.research.google.com/github/danoff/lending-a-hand-in-bringing-ballcarriers-down/blob/main/Run_regressions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Check the versions of libraries

# Python version
import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy
print('scipy: {}'.format(scipy.__version__))
# numpy
import numpy as np
print('numpy: {}'.format(np.__version__))
# matplotlib
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
# pandas
import pandas as pd
print('pandas: {}'.format(pd.__version__))
# scikit-learn
import sklearn
print('sklearn: {}'.format(sklearn.__version__))

import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from scipy import stats
from statsmodels.compat import lzip
import statsmodels
import matplotlib.pyplot as plt

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import cross_val_score
from sklearn import metrics

%matplotlib inline
import seaborn as sns

# Load libraries

from pandas.plotting import scatter_matrix
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
scipy: 1.11.4
numpy: 1.25.2
matplotlib: 3.7.1
pandas: 2.0.3
sklearn: 1.2.2


In [ ]:
# read CSV of analytics staff data

data = pd.read_csv('speed_dist_data_20231228.csv', header=0)
data = data.dropna()

In [ ]:
# Look at the variable names

variablenames = data.keys()
print (variablenames)

Index(['gameId', 'playId', 'frameId', 'playType', 'ballCarrierNflId',
       'ballCarrierDisplayName', 'ballCarrierJerseyNumber', 'ballCarrierSpeed',
       'toPlayerNflId', 'toPlayerDisplayName', 'toPlayerDistanceToBallCarrier',
       'toPlayerAction_tackle', 'toPlayerAction_assist',
       'toPlayerAction_forcedFumble', 'toPlayerAction_missedTackle'],
      dtype='object')


In [ ]:
# describe Staff and WIns

data.describe()

,gameId,playId,frameId,ballCarrierNflId,ballCarrierJerseyNumber,ballCarrierSpeed,toPlayerNflId,toPlayerDistanceToBallCarrier,toPlayerAction_tackle,toPlayerAction_assist,toPlayerAction_forcedFumble,toPlayerAction_missedTackle
count,3.234680e+05,323468.000000,323468.000000,323468.000000,323468.000000,323468.000000,323468.000000,323468.000000,323468.000000,323468.000000,323468.000000,323468.000000
mean,2.022091e+09,2111.400426,25.584280,47904.572103,27.103006,3.647156,47027.589542,11.262820,0.071454,0.043572,0.000819,0.023662
std,9.731683e+01,1276.912692,17.552216,4514.845850,21.469172,2.539941,4781.293551,8.721759,0.257582,0.204140,0.028611,0.151995
min,2.022091e+09,56.000000,1.000000,33084.000000,1.000000,0.000000,35449.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.022091e+09,1008.000000,11.000000,44879.000000,14.000000,1.400000,43641.000000,4.780342,0.000000,0.000000,0.000000,0.000000
50%,2.022091e+09,2130.000000,23.000000,47834.000000,26.000000,3.600000,46188.000000,9.158995,0.000000,0.000000,0.000000,0.000000
75%,2.022091e+09,3126.000000,38.000000,52449.000000,30.000000,5.570000,52458.000000,15.980900,0.000000,0.000000,0.000000,0.000000
max,2.022091e+09,5096.000000,114.000000,55042.000000,89.000000,10.490000,55173.000000,77.950100,1.000000,1.000000,1.000000,1.000000


In [6]:
data.head()

,gameId,playId,frameId,playType,ballCarrierNflId,ballCarrierDisplayName,ballCarrierJerseyNumber,ballCarrierSpeed,toPlayerNflId,toPlayerDisplayName,toPlayerDistanceToBallCarrier,toPlayerAction_tackle,toPlayerAction_assist,toPlayerAction_forcedFumble,toPlayerAction_missedTackle
1,2022090800,56.0,1.0,Completion,42489.0,Stefon Diggs,14.0,6.23,38577.0,Bobby Wagner,11.01360,0.0,0.0,0.0,0.0
2,2022090800,56.0,1.0,Completion,42489.0,Stefon Diggs,14.0,6.23,41239.0,Aaron Donald,12.42790,0.0,0.0,0.0,0.0
3,2022090800,56.0,1.0,Completion,42489.0,Stefon Diggs,14.0,6.23,42816.0,Troy Hill,31.57940,0.0,0.0,0.0,0.0
4,2022090800,56.0,1.0,Completion,42489.0,Stefon Diggs,14.0,6.23,43294.0,Jalen Ramsey,3.63629,1.0,0.0,0.0,0.0
5,2022090800,56.0,1.0,Completion,42489.0,Stefon Diggs,14.0,6.23,43298.0,Leonard Floyd,12.25040,0.0,0.0,0.0,0.0


In [7]:
df = data

In [8]:
df = df[df['toPlayerNflId'] != 0]

In [9]:
df.to_csv('updated_file.csv', index=False)

In [10]:
df['LessThanOneYard'] = (df['toPlayerDistanceToBallCarrier'] < 1).astype(int)

In [11]:
df.head()

,gameId,playId,frameId,playType,ballCarrierNflId,ballCarrierDisplayName,ballCarrierJerseyNumber,ballCarrierSpeed,toPlayerNflId,toPlayerDisplayName,toPlayerDistanceToBallCarrier,toPlayerAction_tackle,toPlayerAction_assist,toPlayerAction_forcedFumble,toPlayerAction_missedTackle,LessThanOneYard
1,2022090800,56.0,1.0,Completion,42489.0,Stefon Diggs,14.0,6.23,38577.0,Bobby Wagner,11.01360,0.0,0.0,0.0,0.0,0
2,2022090800,56.0,1.0,Completion,42489.0,Stefon Diggs,14.0,6.23,41239.0,Aaron Donald,12.42790,0.0,0.0,0.0,0.0,0
3,2022090800,56.0,1.0,Completion,42489.0,Stefon Diggs,14.0,6.23,42816.0,Troy Hill,31.57940,0.0,0.0,0.0,0.0,0
4,2022090800,56.0,1.0,Completion,42489.0,Stefon Diggs,14.0,6.23,43294.0,Jalen Ramsey,3.63629,1.0,0.0,0.0,0.0,0
5,2022090800,56.0,1.0,Completion,42489.0,Stefon Diggs,14.0,6.23,43298.0,Leonard Floyd,12.25040,0.0,0.0,0.0,0.0,0


In [15]:
# Regular regression to ballcarrier speed based on defender distance

model = smf.ols("ballCarrierSpeed ~ LessThanOneYard", data= df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       ballCarrierSpeed   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     3463.
Date:                Sat, 06 Jul 2024   Prob (F-statistic):               0.00
Time:                        17:27:09   Log-Likelihood:            -7.5878e+05
No. Observations:              323468   AIC:                         1.518e+06
Df Residuals:                  323466   BIC:                         1.518e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           3.7050      0.005    814.333      0.000       3.696       3.714
LessThanOneYard    -1.2384      0.021    -58.848      0.000      -1.280      -1.197
==============================================================================
Omnibus:                    50541.896   Durbin-Watson:                   0.025
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            13344.004
Skew:                           0.192   Prob(JB):                         0.00
Kurtosis:                       2.082   Cond. No.                         4.75
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
df = df[df['toPlayerDistanceToBallCarrier'] < 1]

In [17]:
# Regular regression to ballcarrier speed based on defender distance

model = smf.ols("ballCarrierSpeed ~ toPlayerDistanceToBallCarrier", data= df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       ballCarrierSpeed   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     138.4
Date:                Sat, 06 Jul 2024   Prob (F-statistic):           8.14e-32
Time:                        17:29:29   Log-Likelihood:                -31150.
No. Observations:               15120   AIC:                         6.230e+04
Df Residuals:                   15118   BIC:                         6.232e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         1.9801      0.044     44.852      0.000       1.894       2.067
toPlayerDistanceToBallCarrier     0.7824      0.067     11.765      0.000       0.652       0.913
==============================================================================
Omnibus:                     1588.374   Durbin-Watson:                   0.196
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2130.964
Skew:                           0.903   Prob(JB):                         0.00
Kurtosis:                       3.349   Cond. No.                         6.04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""